# hSVM and logistic regression
> Benchmarking two more hyperbolic classifiers

In [1]:
%load_ext autoreload
%autoreload 2

# hSVM and hMLR benchmark:

This code should be run using the `hsvm` conda environment instead of the `hdt` conda environment.

In [2]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# hSVM stuff
sys.path.append("../hsvm")
from hsvm import LinearHSVM

# hLR stuff
sys.path.append("../HyperbolicCV/code")
from lib.lorentz.layers.LMLR import LorentzMLR
from lib.lorentz.manifold import CustomLorentz

import torch

# For benchmarking
# from hyperdt.toy_data import wrapped_normal_mixture
sys.path.append("../HoroRF")
from datasets.gaussian import get_training_data, get_testing_data

# from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import f1_score
import time
from tqdm import tqdm_notebook as tqdm

/home/phil/hdt/notebooks/../hsvm/htools.py:29: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)
/home/phil/hdt/notebooks/../hsvm/htools.py:108: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)
INFO:root:Using numpy backend


In [6]:
# Suppress UserWarning from sklearn and FutureWarning from numba
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
# Train hMLR function


def train_hmlr(X, y, steps=1000):
    # Init class...
    hmlr = LorentzMLR(num_features=X.shape[1], num_classes=2, manifold=CustomLorentz())

    # hMLR outputs logits; labels are {0, 1}
    opt = torch.optim.Adam(hmlr.parameters(), lr=0.01)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for _ in range(steps):
        opt.zero_grad()
        logits = hmlr(X)
        loss = loss_fn(logits[:, 1], y)
        loss.backward()
        opt.step()

    return hmlr

In [7]:
results = pd.DataFrame(columns=["seed", "n_dim", "model", "f1_score", "time"])

seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
dims = [2, 4, 8, 16]
my_tqdm = tqdm(total=len(seeds) * len(dims) * 2 * 5)

for n_dim in dims:
    for seed in seeds:
        # print(n_dim, seed)
        my_tqdm.set_description(f"{n_dim}, {seed}")
        X, y = get_training_data(class_label=n_dim, seed=seed, num_samples=int(800 / 0.8), convert_to_poincare=False)

        # Both models like hyperboloids, so this is easy
        folds = KFold(n_splits=5, shuffle=True, random_state=seed)

        for train_index, test_index in folds.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            X_train_torch = torch.tensor(X_train, dtype=torch.float)
            y_train_torch = torch.tensor(y_train, dtype=torch.float)
            X_test_torch = torch.tensor(X_test, dtype=torch.float)
            y_test_torch = torch.tensor(y_test, dtype=torch.float)

            # hSVM
            t1 = time.time()
            hsvm = LinearHSVM()
            hsvm.fit(X_train, y_train)
            y_pred = hsvm.predict(X_test)
            t2 = time.time()
            results.loc[len(results)] = [seed, n_dim, "hSVM", f1_score(y_test, y_pred, average="micro"), t2 - t1]
            my_tqdm.update()

            # hMLR
            t1 = time.time()
            hmlr = train_hmlr(X_train_torch, y_train_torch)
            y_pred = hmlr(X_test_torch).argmax(dim=1).clone().detach().numpy()
            t2 = time.time()
            results.loc[len(results)] = [seed, n_dim, "hMLR", f1_score(y_test, y_pred, average="micro"), t2 - t1]
            my_tqdm.update()

results.to_csv("../data/processed/hsvm_hmlr_results.csv")

In [9]:
results.groupby(["model", "n_dim"]).mean()["f1_score"] * 100

# Compare to horoDT: 
#   2   91.88
#   4   99.30
#   8   99.96
#  16   100.00

model  n_dim
hMLR   2        0.904000
       4        0.991375
       8        0.999875
       16       1.000000
hSVM   2        0.514875
       4        0.435000
       8        0.464000
       16       0.460000
Name: f1_score, dtype: float64